<a href="https://colab.research.google.com/github/kolatejaswi/Tejaswi-Introduction-to-Gen-AI-and-simple-LLM-inference-on-CPU-finetuning-to-create-custom-chatbot/blob/main/LLM%20inference%20on%20CPU%20and%20finetuning%20model%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (12

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
import pandas as pd
from datasets import load_dataset
from transformers import GPT2Tokenizer

# Load the Persona-Chat dataset
df = pd.read_csv('/content/gdrive/My Drive/personality.csv')

# Check the column names
print(df.columns)

Index(['Unnamed: 0', 'Persona', 'chat'], dtype='object')


In [ ]:
from datasets import load_dataset
import pandas as pd

# Load the Persona-Chat dataset
df=pd.read_csv('/content/gdrive/My Drive/personality.csv')
# For demonstration, let's assume a simple 80/20 split
split_index = int(0.8 * len(df))
df_train = df[:split_index]
df_validation = df[split_index:]

# Display the DataFrames
print(df_train.head())
print(df_validation.head())


   Unnamed: 0                                            Persona  \
0           0   i like to remodel homes. i like to go hunting...   
1           1   my mom is my best friend. i have four sisters...   
2           2   i had a gig at local theater last night. i wo...   
3           3   i am very athletic. i wear contacts. i have b...   
4           4   i am primarily a meat eater. i am a guitar pl...   

                                                chat  
0  hi , how are you doing ? i am getting ready to...  
1  hi , how are you doing today ?\ni am spending ...  
2  we all live in a yellow submarine , a yellow s...  
3  hi ! i work as a gourmet cook .\ni do not like...  
4  how are you doing today\nwhat do you do for ca...  
      Unnamed: 0                                            Persona  \
7151        7151   i believe in ghosts and have seen them on sev...   
7152        7152   i am a blackjack dealer. i graduated with a s...   
7153        7153   i have an iphone. i travel of

In [ ]:
# Install necessary libraries
!pip install datasets transformers torch pandas

# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

# Load and preprocess the dataset
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl 

Mounted at /content/gdrive


In [ ]:
from transformers import GPT2Tokenizer

# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token # Use the end-of-sequence token as the padding token

# Tokenize the training and validation datasets
def tokenize_function(examples):
    return tokenizer(examples["Persona"], padding="max_length", truncation=True)

df_train["tokens"] = df_train["Persona"].apply(lambda x: tokenizer(x, padding="max_length", truncation=True))
df_validation["tokens"] = df_validation["Persona"].apply(lambda x: tokenizer(x, padding="max_length", truncation=True))

# Display tokenized data
print(df_train.head())
print(df_validation.head())


<ipython-input-31-e0e2fe30f627>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["tokens"] = df_train["Persona"].apply(lambda x: tokenizer(x, padding="max_length", truncation=True))


   Unnamed: 0                                            Persona  \
0           0   i like to remodel homes. i like to go hunting...   
1           1   my mom is my best friend. i have four sisters...   
2           2   i had a gig at local theater last night. i wo...   
3           3   i am very athletic. i wear contacts. i have b...   
4           4   i am primarily a meat eater. i am a guitar pl...   

                                                chat  \
0  hi , how are you doing ? i am getting ready to...   
1  hi , how are you doing today ?\ni am spending ...   
2  we all live in a yellow submarine , a yellow s...   
3  hi ! i work as a gourmet cook .\ni do not like...   
4  how are you doing today\nwhat do you do for ca...   

                        tokens  
0  [input_ids, attention_mask]  
1  [input_ids, attention_mask]  
2  [input_ids, attention_mask]  
3  [input_ids, attention_mask]  
4  [input_ids, attention_mask]  
      Unnamed: 0                                        

<ipython-input-31-e0e2fe30f627>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_validation["tokens"] = df_validation["Persona"].apply(lambda x: tokenizer(x, padding="max_length", truncation=True))


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class ConversationDataset(Dataset):
    def __init__(self, df):
        self.input_ids = df["tokens"].apply(lambda x: x["input_ids"]).tolist()
        self.attention_masks = df["tokens"].apply(lambda x: x["attention_mask"]).tolist()

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.input_ids[idx]),
            "attention_mask": torch.tensor(self.attention_masks[idx])
        }

# Create DataLoader for the training and validation datasets
train_dataset = ConversationDataset(df_train)
validation_dataset = ConversationDataset(df_validation)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=8)


In [ ]:

from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

# Load the GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")



In [ ]:
 #Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=1,  # Adjust as needed
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    evaluation_strategy="epoch",
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

# Start training
trainer.train()


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
# Set the model to evaluation mode
model.eval()

# Define a function to generate responses
def generate_response(prompt, model, tokenizer, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs.input_ids, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "Hello! How are you?"
response = generate_response(prompt, model, tokenizer)
print(response)


Hello! How are you?

I'm a little bit nervous. I'm not sure if I'm going to be able to do this, but I'm going to be able to do it. I'm going to be able to do it


In [ ]:
prompt = "what are you doing?"
response = generate_response(prompt, model, tokenizer)
print(response)

what are you doing?

I'm doing a lot of work on my own. I'm doing a lot of work on my own. I'm doing a lot of work on my own. I'm doing a lot of work on my own


In [ ]:
prompt = "do you help me?"
response = generate_response(prompt, model, tokenizer)
print(response)

do you help me?

I'm not sure what to do. I'm not sure what to do.

I'm not sure what to do.

I'm not sure what to do.

I'm not sure what


In [ ]:
prompt = "hi,how are you doing ?"
response = generate_response(prompt, model, tokenizer)
print(response)

hi,how are you doing?

I'm doing fine.

I'm doing fine.

I'm doing fine.

I'm doing fine.

I'm doing fine.

I'm doing fine.



In [ ]:
# Define a function to generate responses
def generate_response(prompt, model, tokenizer, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    # Adjust temperature during generation
    outputs = model.generate(inputs.input_ids, max_length=max_length, temperature=0.7, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "Hi,are you sad?"
response = generate_response(prompt, model, tokenizer,max_length=10)
print(response)

Hi,are you sad? I'm not sad


In [ ]:
# Define a function to generate responses
def generate_response(prompt, model, tokenizer, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    # Adjust temperature during generation
    outputs = model.generate(inputs.input_ids, max_length=max_length, temperature=0.7, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "Hi,can you tell about yourself?"
response = generate_response(prompt, model, tokenizer)
print(response)

Hi,can you tell about yourself?

I'm a writer, and I'm a writer. I'm a writer. I'm a writer. I'm a writer. I'm a writer. I'm a writer. I'm a writer


In [ ]:
# Define a function to generate responses
def generate_response(prompt, model, tokenizer, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    # Adjust temperature during generation
    outputs = model.generate(inputs.input_ids, max_length=max_length, temperature=0.7, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "Hi,what is your work?"
response = generate_response(prompt, model, tokenizer)
print(response)

Hi,what is your work?

I'm a programmer, I'm a designer, I'm a writer, I'm a writer, I'm a writer, I'm a writer, I'm a writer, I'm a writer, I


In [ ]:
# Define a function to generate responses
def generate_response(prompt, model, tokenizer, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    # Adjust temperature during generation
    outputs = model.generate(inputs.input_ids, max_length=max_length, temperature=0.7, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "Hi,what is a machine learning?"
response = generate_response(prompt, model, tokenizer)
print(response)


Hi,what is a machine learning?

Machine learning is a new field of research that has been around for a while. It is a new field of research that has been around for a while. It is a new field of research that has


In [ ]:
# Define a function to generate responses
def generate_response(prompt, model, tokenizer, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    # Adjust temperature during generation
    outputs = model.generate(inputs.input_ids, max_length=max_length, temperature=0.7, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "Hi,what is full form of llm?"
response = generate_response(prompt, model, tokenizer)
print(response)

Hi,what is full form of llm?

I'm not sure what it is, but it's a bit of a mystery.

I'm not sure what it is, but it's a bit of a mystery.




In [ ]:
def generate_response(prompt, model, tokenizer, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt")
    # Adjust temperature during generation
    outputs = model.generate(inputs.input_ids, max_length=max_length, temperature=0.7, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "Hi,what is forest?"
response = generate_response(prompt, model, tokenizer)
print(response)

Hi,what is forest?

Forest is a term used to describe a forest that is not a forest. It is a place where trees grow and where people live. It is a place where people live. It is a place where people live


In [ ]:
#customer support on twitter datsetimport pandas as pd
from transformers import GPT2Tokenizer
df = pd.read_csv('/content/gdrive/My Drive/sample.csv')
print(df.head())

# Tokenize text
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# Preprocess and tokenize your data here


   tweet_id     author_id  inbound                      created_at  \
0    119237        105834     True  Wed Oct 11 06:55:44 +0000 2017   
1    119238  ChaseSupport    False  Wed Oct 11 13:25:49 +0000 2017   
2    119239        105835     True  Wed Oct 11 13:00:09 +0000 2017   
3    119240  VirginTrains    False  Tue Oct 10 15:16:08 +0000 2017   
4    119241        105836     True  Tue Oct 10 15:17:21 +0000 2017   

                                                text response_tweet_id  \
0  @AppleSupport causing the reply to be disregar...            119236   
1  @105835 Your business means a lot to us. Pleas...               NaN   
2  @76328 I really hope you all change but I'm su...            119238   
3  @105836 LiveChat is online at the moment - htt...            119241   
4  @VirginTrains see attached error message. I've...            119243   

   in_response_to_tweet_id  
0                      NaN  
1                 119239.0  
2                      NaN  
3                 

In [ ]:
from transformers import GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained(model)

# Tokenize and prepare dataset
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/gdrive/My Drive/sample.csv',  # Replace with your dataset file path
    block_size=128  # Adjust block size as needed
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# Training arguments
training_args = TrainingArguments(
    output_dir='./gpt2_customer_support_output',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

# Start training
trainer.train()


RuntimeError: Failed to import transformers.data.datasets because of the following error (look up to see its traceback):
Failed to import transformers.models.ernie_m.configuration_ernie_m because of the following error (look up to see its traceback):
No module named 'transformers.models.ernie_m.configuration_ernie_m'

In [ ]:
# Example of generating output
prompt = "Customer asks: what is this product?"
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Generate response
output = model.generate(input_ids, max_length=100)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated Response:", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Response: Customer asks: what is this product?

Answer:

The product is a simple, yet effective, way to help you manage your finances.

It's a simple, yet effective, way to help you manage your finances.

It's a simple, yet effective, way to help you manage your finances.

It's a simple, yet effective, way to help you manage your finances.

It's a simple, yet effective, way to help you manage
